<a href="https://colab.research.google.com/github/danmartin25/Hockey_Model/blob/main/Betting%20Results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import Dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Import Sportsbook data for the 2019-2020 season.
nhl_lines_1920_raw = pd.read_csv('https://raw.githubusercontent.com/danmartin25/Hockey_Model/main/Odds%20Data/nhl%20odds%202019-20.csv')
#nhl_lines_1920_raw.head()

In [3]:
#Reduce data for CLV of Sportsbooks
nhl_lines_1920 = nhl_lines_1920_raw.loc[:,['Date','Team','Close']]
#nhl_lines_1920.head()

In [4]:
#Change Close column to float types
nhl_lines_1920.Close.astype(float)

0       270.0
1      -310.0
2       135.0
3      -150.0
4       116.0
        ...  
2419    135.0
2420    150.0
2421   -165.0
2422   -170.0
2423    155.0
Name: Close, Length: 2424, dtype: float64

In [5]:
#Make a column for the vig of all teams

#Make a condition for when Underdog Odds
nhl_lines_1920.loc[nhl_lines_1920['Close'] >= 0, 'Win %'] = 1/((nhl_lines_1920['Close']/100)+1)
#Make a condition for when Favorite odds
nhl_lines_1920.loc[nhl_lines_1920['Close'] <= 0, 'Win %'] = 1/(1+(-100/nhl_lines_1920['Close']))

In [6]:
#Add column for Log Loss
nhl_lines_1920['Log Loss'] = -np.log(1-(1-nhl_lines_1920['Win %']))

In [7]:
#Make a table for betting lines for each team during the 2019-2020 season.
Avalanche_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Colorado']
Blackhawks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Chicago']
Blue_Jackets_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Columbus']
Blues_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='St.Louis']
Bruins_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Boston']
Canadiens_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Montreal']
Canucks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Vancouver']
Capitals_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Washington']
Coyotes_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Arizona']
Devils_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NewJersey']
Ducks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Anaheim']
Flames_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Calgary']
Flyers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Philadelphia']
Hurricanes_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Carolina']
Islanders_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NYIslanders']
Jets_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Winnipeg']
Kings_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='LosAngeles']
Knights_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Vegas']
Leafs_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Toronto']
Lightning_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='TampaBay']
Oilers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Edmonton']
Panthers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Florida']
Penguins_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Pittsburgh']
Predators_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Nashville']
Rangers_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='NYRangers']
Red_Wings_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Detroit']
Sabres_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Buffalo']
Senators_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Ottawa']
Sharks_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='SanJose']
Stars_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Dallas']
Wild_1920 = nhl_lines_1920.loc[nhl_lines_1920.Team=='Minnesota']

In [8]:
#Reset index for Team tables
Avalanche_1920.reset_index(inplace=True)
Blackhawks_1920.reset_index(inplace=True)
Blue_Jackets_1920.reset_index(inplace=True)
Blues_1920.reset_index(inplace=True)
Bruins_1920.reset_index(inplace=True)
Canadiens_1920.reset_index(inplace=True)
Canucks_1920.reset_index(inplace=True)
Capitals_1920.reset_index(inplace=True)
Coyotes_1920.reset_index(inplace=True)
Devils_1920.reset_index(inplace=True)
Ducks_1920.reset_index(inplace=True)
Flames_1920.reset_index(inplace=True)
Flyers_1920.reset_index(inplace=True)
Hurricanes_1920.reset_index(inplace=True)
Islanders_1920.reset_index(inplace=True)
Jets_1920.reset_index(inplace=True)
Kings_1920.reset_index(inplace=True)
Knights_1920.reset_index(inplace=True)
Leafs_1920.reset_index(inplace=True)
Lightning_1920.reset_index(inplace=True)
Oilers_1920.reset_index(inplace=True)
Panthers_1920.reset_index(inplace=True)
Penguins_1920.reset_index(inplace=True)
Predators_1920.reset_index(inplace=True)
Rangers_1920.reset_index(inplace=True)
Red_Wings_1920.reset_index(inplace=True)
Sabres_1920.reset_index(inplace=True)
Senators_1920.reset_index(inplace=True)
Sharks_1920.reset_index(inplace=True)
Stars_1920.reset_index(inplace=True)
Wild_1920.reset_index(inplace=True)

In [9]:
#Restrict data to just regular season games.
Avalanche_1920 = Avalanche_1920.iloc[:70]
Blackhawks_1920 = Blackhawks_1920.iloc[:70]
Blue_Jackets_1920 = Blue_Jackets_1920.iloc[:70]
Blues_1920 = Blues_1920.iloc[:71]
Bruins_1920 = Bruins_1920.iloc[:70]
Canadiens_1920 = Canadiens_1920.iloc[:71]
Canucks_1920 = Canucks_1920.iloc[:69]
Capitals_1920 = Capitals_1920.iloc[:69]
Coyotes_1920 = Coyotes_1920.iloc[:70]
Devils_1920 = Devils_1920.iloc[:69]
Ducks_1920 = Ducks_1920.iloc[:71]
Flames_1920 = Flames_1920.iloc[:70]
Flyers_1920 = Flyers_1920.iloc[:69]
Hurricanes_1920 = Hurricanes_1920.iloc[:68]
Islanders_1920 = Islanders_1920.iloc[:68]
Jets_1920 = Jets_1920.iloc[:71]
Kings_1920 = Kings_1920.iloc[:70]
Knights_1920 = Knights_1920.iloc[:71]
Leafs_1920 = Leafs_1920.iloc[:70]
Lightning_1920 = Lightning_1920.iloc[:70]
Oilers_1920 = Oilers_1920.iloc[:71]
Panthers_1920 = Panthers_1920.iloc[:69]
Penguins_1920 = Penguins_1920.iloc[:69]
Predators_1920 = Predators_1920.iloc[:69]
Rangers_1920 = Rangers_1920.iloc[:70]
Red_Wings_1920 = Red_Wings_1920.iloc[:71]
Sabres_1920 = Sabres_1920.iloc[:69]
Senators_1920 = Senators_1920.iloc[:71]
Sharks_1920 = Sharks_1920.iloc[:70]
Stars_1920 = Stars_1920.iloc[:69]
Wild_1920 = Wild_1920.iloc[:69]

In [10]:
#Drop columns not needed
Avalanche_1920 = Avalanche_1920.drop(columns = ['index','Date','Team','Close','Win %'])
Blackhawks_1920 = Blackhawks_1920.drop(columns = ['index','Date','Team','Close','Win %'])

In [13]:
#Combine tables into one table
full_lines_1920 = pd.concat([Avalanche_1920, Blackhawks_1920], axis=0)
full_lines_1920

,Log Loss
0,0.606136
1,0.510826
2,0.646627
3,0.524524
4,0.810930
...,...
65,0.441833
66,0.659890
67,0.422857
68,0.788457


In [ ]:
#Avalanche_1920 = Avalanche_1920.T
#Avalanche_1920

,0,1,2,3,4,5,6,7,8,9,...,60,61,62,63,64,65,66,67,68,69
index,21,51,111,143,164,184,212,230,260,308,...,1912,1959,1988,2002,2030,2059,2090,2126,2136,2159
Date,1003,1005,1010,1012,1014,1016,1018,1019,1021,1025,...,222,226,228,229,302,304,306,308,309,311
Team,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,...,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado,Colorado
Close,-120,-150,-110,-145,125,-103,118,185,130,156,...,-155,-200,-105,120,-300,-330,-110,-160,-140,-146
Win %,0.545455,0.6,0.52381,0.591837,0.444444,0.507389,0.458716,0.350877,0.434783,0.390625,...,0.607843,0.666667,0.512195,0.454545,0.75,0.767442,0.52381,0.615385,0.583333,0.593496
Log Loss,0.606136,0.510826,0.646627,0.524524,0.81093,0.678477,0.779325,1.047319,0.832909,0.940007,...,0.497838,0.405465,0.66905,0.788457,0.287682,0.264693,0.646627,0.485508,0.538997,0.521725


In [ ]:
#Restrict Avalanche data to just regular season games.
Blackhawks_1920

,index,Date,Team,Close,Win %,Log Loss
0,24,1004,Chicago,107,0.483092,0.727549
1,107,1010,Chicago,100,0.500000,0.693147
2,137,1012,Chicago,-130,0.565217,0.570545
3,167,1014,Chicago,-110,0.523810,0.646627
4,219,1018,Chicago,-135,0.574468,0.554311
...,...,...,...,...,...,...
65,2049,303,Chicago,-180,0.642857,0.441833
66,2077,305,Chicago,-107,0.516908,0.659890
67,2084,306,Chicago,-190,0.655172,0.422857
68,2121,308,Chicago,120,0.454545,0.788457
